In [ ]:
from sklearn.datasets import fetch_20newsgroups

news_bundle = fetch_20newsgroups(
    subset='train',   # only training data
    remove=('headers', 'footers', 'quotes')
)

raw_articles = news_bundle.data[:2000]  # limit size


In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
common_words = set(stopwords.words('english'))

def clean_text_pipeline(content):
    content = re.sub(r'[^a-zA-Z]', ' ', content)
    content = content.lower()
    tokens = content.split() 
    refined_tokens = [token for token in tokens 
                      if token not in common_words and len(token) > 3]
    return " ".join(refined_tokens)

processed_articles = [clean_text_pipeline(text) for text in raw_articles]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LDA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
bow_vectorizer = CountVectorizer(max_df=0.85, min_df=15)
term_matrix = bow_vectorizer.fit_transform(processed_articles)

In [ ]:
lda_model_v2 = LatentDirichletAllocation(
    n_components=10,
    random_state=101
)

In [ ]:
lda_model_v2.fit(term_matrix)

,"n_components n_components: int, default=10Number of topics... versionchanged:: 0.19 ``n_topics`` was renamed to ``n_components``",10
,"doc_topic_prior doc_topic_prior: float, default=NonePrior of document topic distribution `theta`. If the value is None,defaults to `1 / n_components`.In [1]_, this is called `alpha`.",None
,"topic_word_prior topic_word_prior: float, default=NonePrior of topic word distribution `beta`. If the value is None, defaultsto `1 / n_components`.In [1]_, this is called `eta`.",None
,"learning_method learning_method: {'batch', 'online'}, default='batch'Method used to update `_component`. Only used in :meth:`fit` method.In general, if the data size is large, the online update will be muchfaster than the batch update.Valid options:- 'batch': Batch variational Bayes method. Use all training data in each EM update. Old `components_` will be overwritten in each iteration.- 'online': Online variational Bayes method. In each EM update, use mini-batch of training data to update the ``components_`` variable incrementally. The learning rate is controlled by the ``learning_decay`` and the ``learning_offset`` parameters... versionchanged:: 0.20 The default learning method is now ``""batch""``.",'batch'
,"learning_decay learning_decay: float, default=0.7It is a parameter that control learning rate in the online learningmethod. The value should be set between (0.5, 1.0] to guaranteeasymptotic convergence. When the value is 0.0 and batch_size is``n_samples``, the update method is same as batch learning. In theliterature, this is called kappa.",0.7
,"learning_offset learning_offset: float, default=10.0A (positive) parameter that downweights early iterations in onlinelearning. It should be greater than 1.0. In the literature, this iscalled tau_0.",10.0
,"max_iter max_iter: int, default=10The maximum number of passes over the training data (aka epochs).It only impacts the behavior in the :meth:`fit` method, and not the:meth:`partial_fit` method.",10
,"batch_size batch_size: int, default=128Number of documents to use in each EM iteration. Only used in onlinelearning.",128
,"evaluate_every evaluate_every: int, default=-1How often to evaluate perplexity. Only used in `fit` method.set it to 0 or negative number to not evaluate perplexity intraining at all. Evaluating perplexity can help you check convergencein training process, but it will also increase total training time.Evaluating perplexity in every iteration might increase training timeup to two-fold.",-1
,"total_samples total_samples: int, default=1e6Total number of documents. Only used in the :meth:`partial_fit` method.",1000000.0
,"perp_tol perp_tol: float, default=1e-1Perplexity tolerance. Only used when ``evaluate_every`` is greater than 0.",0.1


In [ ]:
lda_perp_score = lda_model_v2.perplexity(term_matrix)
print("LDA Perplexity Score:", lda_perp_score)

LDA Perplexity Score: 943.1316042330686


NMF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [ ]:
tfidf_encoder = TfidfVectorizer(max_df=0.85, min_df=15)
tfidf_features = tfidf_encoder.fit_transform(processed_articles)

In [ ]:
nmf_model_v2 = NMF(
    n_components=10,
    random_state=101,
    init='nndsvd'
)

In [ ]:

nmf_model_v2.fit(tfidf_features)

c:\Users\Admin\AppData\Local\Python\pythoncore-3.11-64\Lib\site-packages\sklearn\decomposition\_nmf.py:1720: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


,"n_components n_components: int or {'auto'} or None, default='auto'Number of components. If `None`, all features are kept.If `n_components='auto'`, the number of components is automatically inferredfrom W or H shapes... versionchanged:: 1.4 Added `'auto'` value... versionchanged:: 1.6 Default value changed from `None` to `'auto'`.",10
,"init init: {'random', 'nndsvd', 'nndsvda', 'nndsvdar', 'custom'}, default=NoneMethod used to initialize the procedure.Valid options:- `None`: 'nndsvda' if n_components <= min(n_samples, n_features), otherwise random.- `'random'`: non-negative random matrices, scaled with: `sqrt(X.mean() / n_components)`- `'nndsvd'`: Nonnegative Double Singular Value Decomposition (NNDSVD) initialization (better for sparseness)- `'nndsvda'`: NNDSVD with zeros filled with the average of X (better when sparsity is not desired)- `'nndsvdar'` NNDSVD with zeros filled with small random values (generally faster, less accurate alternative to NNDSVDa for when sparsity is not desired)- `'custom'`: Use custom matrices `W` and `H` which must both be provided... versionchanged:: 1.1 When `init=None` and n_components is less than n_samples and n_features defaults to `nndsvda` instead of `nndsvd`.",'nndsvd'
,"solver solver: {'cd', 'mu'}, default='cd'Numerical solver to use:- 'cd' is a Coordinate Descent solver.- 'mu' is a Multiplicative Update solver... versionadded:: 0.17 Coordinate Descent solver... versionadded:: 0.19 Multiplicative Update solver.",'cd'
,"beta_loss beta_loss: float or {'frobenius', 'kullback-leibler', 'itakura-saito'}, default='frobenius'Beta divergence to be minimized, measuring the distance between Xand the dot product WH. Note that values different from 'frobenius'(or 2) and 'kullback-leibler' (or 1) lead to significantly slowerfits. Note that for beta_loss <= 0 (or 'itakura-saito'), the inputmatrix X cannot contain zeros. Used only in 'mu' solver... versionadded:: 0.19",'frobenius'
,"tol tol: float, default=1e-4Tolerance of the stopping condition.",0.0001
,"max_iter max_iter: int, default=200Maximum number of iterations before timing out.",200
,"random_state random_state: int, RandomState instance or None, default=NoneUsed for initialisation (when ``init`` == 'nndsvdar' or'random'), and in Coordinate Descent. Pass an int for reproducibleresults across multiple function calls.See :term:`Glossary `.",101
,"alpha_W alpha_W: float, default=0.0Constant that multiplies the regularization terms of `W`. Set it to zero(default) to have no regularization on `W`... versionadded:: 1.0",0.0
,"alpha_H alpha_H: float or ""same"", default=""same""Constant that multiplies the regularization terms of `H`. Set it to zero tohave no regularization on `H`. If ""same"" (default), it takes the same value as`alpha_W`... versionadded:: 1.0",'same'
,"l1_ratio l1_ratio: float, default=0.0The regularization mixing parameter, with 0 <= l1_ratio <= 1.For l1_ratio = 0 the penalty is an elementwise L2 penalty(aka Frobenius Norm).For l1_ratio = 1 it is an elementwise L1 penalty.For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2... versionadded:: 0.17 Regularization parameter *l1_ratio* used in the Coordinate Descent solver.",0.0
,"verbose verbose: int, default=0Whether to be verbose.",0


Coherence Score

In [ ]:
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
tokenized_corpus = [doc.split() for doc in processed_articles]

In [ ]:
vocab_dictionary = Dictionary(tokenized_corpus)
bow_corpus = [vocab_dictionary.doc2bow(text) for text in tokenized_corpus]

In [ ]:
coherence_eval = CoherenceModel(
    topics=[[bow_vectorizer.get_feature_names_out()[i] 
            for i in topic.argsort()[-10:]]
            for topic in lda_model_v2.components_],
    texts=tokenized_corpus,
    dictionary=vocab_dictionary,
    coherence='c_v'
)

In [ ]:
print("Coherence Score:", coherence_eval.get_coherence())

Coherence Score: 0.5082340337731592


pyLDAvis Visualization

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
lda_visual = pyLDAvis.lda_model.prepare(
    lda_model_v2,
    term_matrix,
    bow_vectorizer
)

In [ ]:
lda_visual

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.144315  0.020636       1        1  18.593830
0     -0.141618  0.022847       2        1  14.858011
4     -0.042569 -0.151739       3        1  14.838600
6     -0.138293 -0.052579       4        1  12.655757
9      0.059750 -0.074904       5        1   8.543040
5      0.236735 -0.018919       6        1   8.014888
7      0.058759  0.160981       7        1   7.857616
2      0.113424 -0.100288       8        1   6.812925
3     -0.164816  0.086198       9        1   4.858457
8      0.162943  0.107767      10        1   2.966876, topic_info=           Term        Freq       Total Category  logprob  loglift
653       jesus  230.000000  230.000000  Default  30.0000  30.0000
484        file  255.000000  255.000000  Default  29.0000  29.0000
902      output  204.000000  204.000000  Default  28.0000  28.0000
1405    version  183.000000  183.000000  Default  27.0000  27.0000
1445    windows  229.000000  229.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
1391       used   20.456709  286.425821  Topic10  -4.8632   0.8785
1077  religious   13.329096   59.892955  Topic10  -5.2916   2.0151
717        list   15.868243  148.789595  Topic10  -5.1172   1.2794
488        find   16.422131  237.087867  Topic10  -5.0829   0.8479
1295     system   16.392557  315.040069  Topic10  -5.0847   0.5618

[573 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2         1  0.935431  absolute
2         2  0.025282  absolute
2         8  0.025282  absolute
7         3  0.335255    access
7         4  0.049667    access
...     ...       ...       ...
1474      1  0.045951      york
1474      2  0.114878      york
1474      4  0.068927      york
1474      7  0.689269      york
1474      8  0.068927      york

[2113 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 5, 7, 10, 6, 8, 3, 4, 9])

BERTopic

In [ ]:
from bertopic import BERTopic

c:\Users\Admin\AppData\Local\Python\pythoncore-3.11-64\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
semantic_topic_engine = BERTopic(verbose=True)
topic_labels, topic_probs = semantic_topic_engine.fit_transform(processed_articles)

2026-02-18 12:06:54,730 - BERTopic - Embedding - Transforming documents to embeddings.
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 550.98it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 63/63 [00:32<00:00,  1.91it/s]
2026-02-18 12:07:31,861 - BERTopic - Embedding - Completed ✓
2026-02-18 12:07:31,861 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-02-18 12:07:46,002 - BERTopic - Dimensionality - Completed ✓
2026-02-18 12:07:46,004 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-02-18 12:07:46,122 - BERTopic - Cluster - Completed ✓
2026-02-18 12:07:46,122 - B

In [ ]:
semantic_topic_engine.visualize_topics()